In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.4 \n'

In [2]:
ch = R.TChain("MiniDST")
data_mod = "MC_mod2"
data_source="/data/HPS/data/physrun2021/" + data_mod
energy="*"
particle="e-"
if particle == "e+":
    particle_s = "e\+"
else:
    particle_s = particle
if energy == "*":
    energy_s = "all_"
else:
    energy_s = energy;
ch.Add(data_source+"/hpsForward_"+particle_s+"_"+energy+"GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")
df = R.RDataFrame(ch)

Number of events loaded:  11.079M


In [3]:
dfx = df.Define("n_kf_track","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n") \
    .Filter("n_kf_track==1").Filter("ecal_cluster_energy.size()==1") \
    .Define("trk_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]==1) return(i);} return(-1);") \
    .Define("diff_posx_ecal","ecal_cluster_x[0] - track_x_at_ecal[trk_idx]") \
    .Define("diff_posy_ecal","ecal_cluster_y[0] - track_y_at_ecal[trk_idx]") \
    .Define("diff_E_p","part_energy[0] - get_vec_abs(part_px,part_py,part_pz)[0]") \
    .Define("diff_p_kf_gbl","get_vec_abs(part_px,part_py,part_pz)[0] - get_vec_abs(part_px,part_py,part_pz)[1]") \
    .Define("mc_part_primary_z","find_end_z_of_primary_mc_part(mc_part_z, mc_part_end_z)") \
    .Define("mc_part_daughters_z","find_average_end_of_daughters_of_primary_mc_part(mc_part_z, mc_part_end_z, mc_part_daughters)")
cnt  = df.Count()
cntx = dfx.Count()

In [4]:
hx_diff_posx_ecal=dfx.Histo1D(("hx_diff_posx","Diff Ecal_x - Track_x;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_top=dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_posx_top","Diff Ecal_x - Track_x Top;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_bot=dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_posx_bot","Diff Ecal_x - Track_x Bot;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_L=dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_posx_L","Diff Ecal_x - Track_x L;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_R=dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_posx_R","Diff Ecal_x - Track_x R;#Delta x[mm]",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posy_ecal=dfx.Histo1D(("hx_diff_posy","Diff Ecal_y - Track_y;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_top=dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_posy_top","Diff Ecal_y - Track_y Top;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_bot=dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_posy_bot","Diff Ecal_y - Track_y Bot;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_L=dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_posy_L","Diff Ecal_y - Track_y L;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_R=dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_posy_R","Diff Ecal_y - Track_y R;#Delta y[mm]",1000,-20.,20.),"diff_posy_ecal")
hx_diff_e_p = dfx.Histo1D(("hx_diff_e_p", "Diff Energy - Momentum;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_top = dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_e_p_top", "Diff Energy - Momentum Top;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_bot = dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_e_p_bot", "Diff Energy - Momentum Bot;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_L = dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_e_p_L", "Diff Energy - Momentum Left;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_R = dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_e_p_R", "Diff Energy - Momentum Right;#Delta E[GeV]", 1000, -3., 1.), "diff_E_p")
hx_diff_p_kf_gbl = dfx.Histo1D(("hx_diff_p_kf_gbl","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_top = dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_p_kf_gbl_top","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_bot = dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_p_kf_gbl_bot","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_L = dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_p_kf_gbl_L","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_R = dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_p_kf_gbl_R","Diff P_kf - P_gbl;#Delta P[GeV]",1000,-0.2,0.2),"diff_p_kf_gbl")

hx_mc_part_primary_z = dfx.Histo1D(("hx_mc_part_primary_z","End of track for primary MC particle.;z[mm]",1000,-0.1,1700.),"mc_part_primary_z")
hx_mc_part_ave_daughter_z = dfx.Histo1D(("hx_mc_part_ave_daughter_z","Average end of track for primary MC particle daughters.;z[mm]",1000,-0.1,1700.),"mc_part_daughters_z")


In [5]:
cc4 = R.TCanvas("cc4","Canvas",1200,600)
cc4.Divide(2,2)
p1=cc4.cd(1)
#p1.SetLogy()
hx_diff_posx_ecal.Draw()
hx_diff_posx_ecal_top.SetLineColor(R.kGreen)
hx_diff_posx_ecal_top.Draw("same")
hx_diff_posx_ecal_bot.SetLineColor(R.kRed)
hx_diff_posx_ecal_bot.Draw("same")
hx_diff_posx_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posx_ecal_L.Draw("same")
hx_diff_posx_ecal_R.SetLineColor(R.kCyan)
hx_diff_posx_ecal_R.Draw("same")
p2=cc4.cd(2)
#p2.SetLogy()
hx_diff_posy_ecal.Draw()
hx_diff_posy_ecal_top.SetLineColor(R.kGreen)
hx_diff_posy_ecal_top.Draw("same")
hx_diff_posy_ecal_bot.SetLineColor(R.kRed)
hx_diff_posy_ecal_bot.Draw("same")
hx_diff_posy_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posy_ecal_L.Draw("same")
hx_diff_posy_ecal_R.SetLineColor(R.kCyan)
hx_diff_posy_ecal_R.Draw("same")
p3=cc4.cd(3)
# p3.SetLogy()
hx_diff_e_p.Draw()
hx_diff_e_p_top.SetLineColor(R.kGreen)
hx_diff_e_p_top.Draw("same")
hx_diff_e_p_bot.SetLineColor(R.kRed)
hx_diff_e_p_bot.Draw("same")
hx_diff_e_p_L.SetLineColor(R.kMagenta)
hx_diff_e_p_L.Draw("same")
hx_diff_e_p_R.SetLineColor(R.kCyan)
hx_diff_e_p_R.Draw("same")
p4=cc4.cd(4)
# p4.SetLogy()
hx_diff_p_kf_gbl.Draw()
hx_diff_p_kf_gbl_top.SetLineColor(R.kGreen)
hx_diff_p_kf_gbl_top.Draw("same")
hx_diff_p_kf_gbl_bot.SetLineColor(R.kRed)
hx_diff_p_kf_gbl_bot.Draw("same")
hx_diff_p_kf_gbl_L.SetLineColor(R.kMagenta)
hx_diff_p_kf_gbl_L.Draw("same")
hx_diff_p_kf_gbl_R.SetLineColor(R.kCyan)
hx_diff_p_kf_gbl_R.Draw("same")

# h_trk_type.Draw()
cc4.cd(0)
leg1 = R.TLegend(0.83,0.8,1.,1.)
leg1.AddEntry(hx_diff_posx_ecal.GetValue(), "All data with one cluster")
leg1.AddEntry(hx_diff_posx_ecal_top.GetValue(),"Top only")
leg1.AddEntry(hx_diff_posx_ecal_bot.GetValue(),"Bottom only")
leg1.AddEntry(hx_diff_posx_ecal_L.GetValue(),"Left Only x<0")
leg1.AddEntry(hx_diff_posx_ecal_R.GetValue(),"Right Only x>0")
leg1.Draw()

cc4.Draw()
cc4.SaveAs(data_mod+"_"+particle+"_"+energy_s+"GeV_compare.pdf")

Info in <TCanvas::Print>: pdf file MC_mod2_e-_all_GeV_compare.pdf has been created


In [6]:
cc5 = R.TCanvas("cc5","Canvas",800,600)
hx_mc_part_ave_daughter_z.SetStats(0)
hx_mc_part_ave_daughter_z.SetLineColor(R.kRed)
hx_mc_part_ave_daughter_z.SetLineWidth(2)
hx_mc_part_ave_daughter_z.Draw()
l1 = R.TLine(1394, 0, 1394, hx_mc_part_ave_daughter_z.GetMaximum())
l1.SetLineWidth(2)
l1.SetLineColor(R.kOrange)

l1.Draw()
hx_mc_part_primary_z.SetLineColor(R.kBlue)
hx_mc_part_primary_z.Draw("same")
leg5 = R.TLegend(0.1,0.7,0.4,0.9)
leg5.AddEntry(hx_mc_part_primary_z.GetValue(),"Endpoint of primary MC particle")
leg5.AddEntry(hx_mc_part_ave_daughter_z.GetValue(),"Average endpoint of daughters.")
leg5.AddEntry(l1,"Ecal face.")
leg5.Draw()
cc5.Draw()

In [16]:
# Link the C++ fit_gaus_tail function to a TF1 so the fitting of the histogram works properly in Python
R.gInterpreter.Declare('''
void setup_gaus_tail(void){
auto gaustail = new TF1("gaustail",fit_gaus_tail,-20.,20.,4);
gaustail->SetParameters(1208, -5, 3.4, 1.);
gaustail->SetParNames("A","Mu","Area","tail");
std::cout << "The gaustail function is now setup. " << std::endl;
}
''')
R.setup_gaus_tail()

# Link the C++ fit_gaus_tail function to a TF1 so the fitting of the histogram works properly in Python
R.gInterpreter.Declare('''
void setup_gaus_tailn(void){
auto gaustailn = new TF1("gaustailn",fit_gaus_tailn,-20.,20.,4);
gaustailn->SetParameters(1208, -5, 3.4, 1.);
gaustailn->SetParNames("A","Mu","Area","tail");
std::cout << "The gaustailn function is now setup. " << std::endl;
}
''')
R.setup_gaus_tailn()

The fitgaus function is now setup. 
The gaustailn function is now setup. 


input_line_251:2:6: error: redefinition of 'setup_gaus_tail'
void setup_gaus_tail(void){
     ^
input_line_168:2:6: note: previous definition is here
void setup_gaus_tail(void){
     ^


In [20]:
cc6 = R.TCanvas("cc6","Canvas",1200,400)
cc6.Divide(2,1)
p1=cc6.cd(1)
#p1.SetLogy()

tophist = hx_diff_posy_ecal_top.GetPtr()
xax = tophist.GetXaxis()
xax.SetLimits(-10,10)
hx_diff_posy_ecal_top.SetLineColor(R.kGreen)
hx_diff_posy_ecal_top.Draw()
hx_diff_posy_ecal_bot.SetLineColor(R.kRed)
hx_diff_posy_ecal_bot.Draw("same")
print("Fit results for the 'top' distribution (green)")
fit_result_T = hx_diff_posy_ecal_top.Fit("gaustail","S")
print("Fit results for the 'bottom' distribution (red)")
fit_result_B = hx_diff_posy_ecal_bot.Fit("gaustail","S")

p2=cc6.cd(2)
#p2.SetLogy()

hx_diff_posx_ecal_R.Draw()
hx_diff_posx_ecal_R.SetLineColor(R.kCyan)
hx_diff_posx_ecal_L.Draw("same")
hx_diff_posx_ecal_L.SetLineColor(R.kMagenta)

print("Fit results for the 'left' distribution (magenta)")
fit_result_L = hx_diff_posx_ecal_L.Fit("gaustailn","S")
print("Fit results for the 'right' distribution (cyan)")
fit_result_R = hx_diff_posx_ecal_R.Fit("gaustail","S")
# h_trk_type.Draw()
cc6.cd(0)
leg1 = R.TLegend(0.83,0.8,1.,1.)
leg1.AddEntry(hx_diff_posx_ecal.GetValue(), "All data with one cluster")
leg1.AddEntry(hx_diff_posx_ecal_top.GetValue(),"Top only")
leg1.AddEntry(hx_diff_posx_ecal_bot.GetValue(),"Bottom only")
leg1.AddEntry(hx_diff_posx_ecal_L.GetValue(),"Left Only x<0")
leg1.AddEntry(hx_diff_posx_ecal_R.GetValue(),"Right Only x>0")
leg1.Draw()

cc6.Draw()

Fit results for the 'top' distribution (green)
Fit results for the 'bottom' distribution (red)
Fit results for the 'left' distribution (magenta)
Fit results for the 'right' distribution (cyan)
 FCN=144992 FROM MIGRAD    STATUS=CONVERGED     692 CALLS         693 TOTAL
                     EDM=9.54956e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.4 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  A            6.88735e+04   3.71040e+01  -1.22999e-01   6.16413e-06
   2  Mu          -4.49208e-01   7.89637e-04   3.55206e-06  -1.01893e+00
   3  Area         6.21628e-01   6.26140e-04  -1.81573e-06   4.44901e-01
   4  tail         1.27159e+00   1.61780e-03  -2.25870e-06   3.80705e-01
 FCN=247312 FROM MIGRAD    STATUS=CONVERGED     534 CALLS         535 TOTAL
                     EDM=5.54959e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.0 per cent
  EXT PARAMETER                  

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc6


In [21]:
Location_z = [1394., 1474., 1430]
top_mean = [0.8491034824579542]
top_mean_e = [0.0005465571955235653]
bot_mean = [-0.675077954037954]
bot_mean_e = [0.0010992656875469878]
left_mean = [-1.7182140998711766]
left_mean_e = [0.0038091590937876793]
right_mean = [3.4865832364916374]
right_mean_e = [0.00171205258079781]

# MC_mod data
top_mean.append(-1.3079245467806906)
top_mean_e.append(0.0008683112270701557)
bot_mean.append(2.0717543424688887)
bot_mean_e.append(0.001122789028803333)
left_mean.append(2.770200054296437)
left_mean_e.append(0.003142993822897123)
right_mean.append(-0.9855838237712853)
right_mean_e.append(0.0030117573934326687)

# MC_mod2 data

top_mean.append(fit_result_T.Parameter(1))
top_mean_e.append(fit_result_T.Error(1))
bot_mean.append(fit_result_B.Parameter(1))
bot_mean_e.append(fit_result_B.Error(1))
left_mean.append(fit_result_L.Parameter(1))
left_mean_e.append(fit_result_L.Error(1))
right_mean.append(fit_result_R.Parameter(1))
right_mean_e.append(fit_result_R.Error(1))

print(f"top_mean.append({top_mean[-1]})")
print(f"top_mean_e.append({top_mean_e[-1]})")
print(f"bot_mean.append({bot_mean[-1]})")
print(f"bot_mean_e.append({bot_mean_e[-1]})")
print(f"left_mean.append({left_mean[-1]})")
print(f"left_mean_e.append({left_mean_e[-1]})")
print(f"right_mean.append({right_mean[-1]})")
print(f"right_mean_e.append({right_mean_e[-1]})")


top_mean.append(-0.4492083028957761)
top_mean_e.append(0.0007896370392242154)
bot_mean.append(0.5361668473019777)
bot_mean_e.append(0.0009602386153652128)
left_mean.append(1.7645701035334438)
left_mean_e.append(0.00344247925241593)
right_mean.append(0.9492128301119738)
right_mean_e.append(0.0021300238539998504)


In [31]:
from numpy.polynomial import Polynomial
pfit_top = Polynomial.fit(Location_z, top_mean, 1)
top_dat = pfit_top.linspace(2)
pfit_bot = Polynomial.fit(Location_z, bot_mean, 1)
bot_dat = pfit_bot.linspace(2)
pfit_left = Polynomial.fit(Location_z, left_mean, 1)
left_dat = pfit_left.linspace(2)
pfit_right = Polynomial.fit(Location_z, right_mean, 1)
right_dat = pfit_right.linspace(2)

In [41]:
cc7 = R.TCanvas("cc7","CC7", 800,600)
h_empty = R.TH2F("h_empty","Crossing of track paths",100,1384,1484,100,-5., 6)
h_empty.SetStats(0)
h_empty.Draw()

top_markers=[]
bot_markers=[]
left_markers=[]
right_markers=[]

for i in range(len(top_mean)):
    top_markers.append(R.TMarker(Location_z[i], top_mean[i],21))
    top_markers[-1].SetMarkerColor(R.kGreen+1)
    top_markers[-1].Draw()
    bot_markers.append(R.TMarker(Location_z[i], bot_mean[i],21))
    bot_markers[-1].SetMarkerColor(R.kRed+1)
    bot_markers[-1].Draw()
    left_markers.append(R.TMarker(Location_z[i], left_mean[i],21))
    left_markers[-1].SetMarkerColor(R.kMagenta+1)
    left_markers[-1].Draw()
    right_markers.append(R.TMarker(Location_z[i], right_mean[i],21))
    right_markers[-1].SetMarkerColor(R.kCyan+1)
    right_markers[-1].Draw()

l_top = R.TLine(top_dat[0][0], top_dat[1][0], top_dat[0][1], top_dat[1][1])
l_top.SetLineColor(R.kGreen)
l_top.SetLineWidth(2)
l_top.Draw()
l_bot = R.TLine(bot_dat[0][0], bot_dat[1][0], bot_dat[0][1], bot_dat[1][1])
l_bot.SetLineColor(R.kRed)
l_bot.SetLineWidth(2)
l_bot.Draw()
l_left = R.TLine(left_dat[0][0], left_dat[1][0], left_dat[0][1], left_dat[1][1])
l_left.SetLineColor(R.kMagenta)
l_left.SetLineWidth(2)
l_left.Draw()
l_right = R.TLine(right_dat[0][0], right_dat[1][0], right_dat[0][1], right_dat[1][1])
l_right.SetLineColor(R.kCyan)
l_right.SetLineWidth(2)
l_right.Draw()
l_zero = R.TLine(1384, 0, 1484, 0)
l_zero.SetLineColor(R.kBlack)
l_zero.Draw()
cc7.cd(0)
leg1 = R.TLegend(0.83,0.8,1.,1.)
leg1.AddEntry(l_top,"Top")
leg1.AddEntry(l_bot,"Bottom")
leg1.AddEntry(l_left,"Left x<0")
leg1.AddEntry(l_right,"Right x>0")
leg1.Draw()

cc7.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc7
Warning in <TROOT::Append>: Replacing existing TH1: h_empty (Potential memory leak).


In [45]:
z=1417
print(f"{pfit_top(z)} {pfit_bot(z)} {pfit_left(z)} {pfit_right(z)}")

0.11547810278450327 0.10603666486162366 0.07890770764983901 2.019040098192633
